In [1]:
!pip install scikit-learn sentence-transformers transformers faiss-cpu pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 76.2 MB/s eta 0:00:00


In [2]:
import pandas as pd

In [11]:
from google.colab import drive


drive.mount('/content/drive')


Mounted at /content/drive


In [15]:
#library fot .parquet
!pip install pyarrow


In [14]:
import os
os.path.exists("/content/drive/MyDrive/Colab Notebooks/food_sample.parquet")


True

In [16]:
import pandas as pd

file_path = "/content/drive/MyDrive/Colab Notebooks/food_sample.parquet"
df = pd.read_parquet(file_path)

# quick inspection
print(df.shape)
print(df.columns)
df.head(3)


(35840, 110)
Index(['additives_n', 'additives_tags', 'allergens_tags', 'brands_tags',
       'brands', 'categories', 'categories_tags', 'categories_properties',
       'checkers_tags', 'ciqual_food_name_tags',
       ...
       'states_tags', 'stores_tags', 'stores', 'traces_tags', 'unique_scans_n',
       'unknown_ingredients_n', 'unknown_nutrients_tags', 'vitamins_tags',
       'with_non_nutritive_sweeteners', 'with_sweeteners'],
      dtype='object', length=110)


,additives_n,additives_tags,allergens_tags,brands_tags,brands,categories,categories_tags,categories_properties,checkers_tags,ciqual_food_name_tags,...,states_tags,stores_tags,stores,traces_tags,unique_scans_n,unknown_ingredients_n,unknown_nutrients_tags,vitamins_tags,with_non_nutritive_sweeteners,with_sweeteners
0,NaN,None,[en:nuts],[xx:bovetti],Bovetti,"Petit-déjeuners,Produits à tartiner,Produits à...","[en:breakfasts, en:spreads, en:sweet-spreads, ...","{'agribalyse_food_code': 31032.0, 'agribalyse_...",[],[chocolate-spread-with-hazelnuts],...,"[en:to-be-completed, en:nutrition-facts-comple...",[],,[],1.0,NaN,[],[],NaN,NaN
1,0.0,[],[],[lagg-s],Lagg's,null,[en:null],"{'agribalyse_food_code': None, 'agribalyse_pro...",[],[unknown],...,"[en:to-be-completed, en:nutrition-facts-comple...",None,<NA>,[],1.0,0.0,[],[],NaN,NaN
2,0.0,[],[],[lagg-s],Lagg's,"Plant-based foods and beverages, Beverages, Ho...","[en:plant-based-foods-and-beverages, en:bevera...","{'agribalyse_food_code': 18020.0, 'agribalyse_...",[],[unknown],...,"[en:to-be-completed, en:nutrition-facts-comple...",None,<NA>,[],NaN,0.0,[],[],NaN,NaN


In [18]:
df.columns.tolist()


['additives_n',
 'additives_tags',
 'allergens_tags',
 'brands_tags',
 'brands',
 'categories',
 'categories_tags',
 'categories_properties',
 'checkers_tags',
 'ciqual_food_name_tags',
 'cities_tags',
 'code',
 'compared_to_category',
 'complete',
 'completeness',
 'correctors_tags',
 'countries_tags',
 'created_t',
 'creator',
 'data_quality_errors_tags',
 'data_quality_info_tags',
 'data_quality_warnings_tags',
 'data_sources_tags',
 'ecoscore_data',
 'ecoscore_grade',
 'ecoscore_score',
 'ecoscore_tags',
 'editors',
 'emb_codes_tags',
 'emb_codes',
 'entry_dates_tags',
 'food_groups_tags',
 'generic_name',
 'images',
 'informers_tags',
 'ingredients_analysis_tags',
 'ingredients_from_palm_oil_n',
 'ingredients_n',
 'ingredients_original_tags',
 'ingredients_percent_analysis',
 'ingredients_tags',
 'ingredients_text',
 'ingredients_with_specified_percent_n',
 'ingredients_with_unspecified_percent_n',
 'ingredients_without_ciqual_codes_n',
 'ingredients_without_ciqual_codes',
 'ingre

In [20]:
import pandas as pd
import json

# Reload the parquet file (optional safety)
file_path = "/content/drive/MyDrive/Colab Notebooks/food_sample.parquet"
df = pd.read_parquet(file_path)

# Function to safely extract numeric nutrients
def safe_get(d, key):
    try:
        return d.get(key, 0) if isinstance(d, dict) else 0
    except:
        return 0

# Build a simplified DataFrame
food_df = pd.DataFrame({
    "product_name": df["product_name"],
    "ingredients_text": df.get("ingredients_text", ""),
    "category": df.get("categories", ""),
    "brand": df.get("brands", ""),
    "nutri_grade": df.get("nutriscore_grade", ""),
    "energy_kcal": df["nutriments"].apply(lambda x: safe_get(x, "energy-kcal_100g")),
    "proteins": df["nutriments"].apply(lambda x: safe_get(x, "proteins_100g")),
    "fat": df["nutriments"].apply(lambda x: safe_get(x, "fat_100g")),
    "sugar": df["nutriments"].apply(lambda x: safe_get(x, "sugars_100g")),
})

# Drop incomplete rows and fill missing text
food_df = food_df.dropna(subset=["product_name"]).fillna("")
food_df["ingredients_text"] = food_df["ingredients_text"].astype(str).str[:300]


food_df.head(3)


,product_name,ingredients_text,category,brand,nutri_grade,energy_kcal,proteins,fat,sugar
0,"[{'lang': 'main', 'text': 'Véritable pâte à ta...",[],"Petit-déjeuners,Produits à tartiner,Produits à...",Bovetti,e,0,0,0,0
1,"[{'lang': 'main', 'text': 'Chamomile Herbal Te...","[{'lang': 'main', 'text': 'CHAMOMILE FLOWERS.'...",null,Lagg's,unknown,0,0,0,0
2,"[{'lang': 'main', 'text': 'Lagg's, herbal tea,...","[{'lang': 'main', 'text': 'Peppermint.'}\n {'l...","Plant-based foods and beverages, Beverages, Ho...",Lagg's,unknown,0,0,0,0


In [21]:
def make_text(row):
    return (
        f"{row['product_name']} ({row['brand']}): "
        f"{row['energy_kcal']} kcal per 100 g | Protein {row['proteins']} g | "
        f"Fat {row['fat']} g | Sugar {row['sugar']} g. "
        f"Category: {row['category']}. "
        f"Nutri-Score: {row['nutri_grade']}. "
        f"Ingredients: {row['ingredients_text']}"
    )

food_df["text"] = food_df.apply(make_text, axis=1)

# Keep only id + text for RAG retriever
food_knowledge = food_df[["text"]].reset_index().rename(columns={"index": "id"})

print("✅ Text generation complete.")
food_knowledge.head(3)


✅ Text generation complete.


,id,text
0,0,"[{'lang': 'main', 'text': 'Véritable pâte à ta..."
1,1,"[{'lang': 'main', 'text': 'Chamomile Herbal Te..."
2,2,"[{'lang': 'main', 'text': ""Lagg's, herbal tea,..."


In [22]:
output_path = "/content/drive/MyDrive/Colab Notebooks/food_knowledge.json"

records = food_knowledge.to_dict(orient="records")
with open(output_path, "w") as f:
    json.dump(records, f, indent=2)

print(f"✅ Saved knowledge base: {output_path}")
print("Total records:", len(records))


✅ Saved knowledge base: /content/drive/MyDrive/Colab Notebooks/food_knowledge.json
Total records: 35840


In [23]:
output_path = "/content/drive/MyDrive/Colab Notebooks/food_knowledge.json"

records = food_knowledge.to_dict(orient="records")
with open(output_path, "w") as f:
    json.dump(records, f, indent=2)

print(f"✅ Saved knowledge base: {output_path}")
print("Total records:", len(records))


✅ Saved knowledge base: /content/drive/MyDrive/Colab Notebooks/food_knowledge.json
Total records: 35840


In [24]:
import os
print("File exists:", os.path.exists(output_path))

# Optional quick peek
with open(output_path) as f:
    sample = json.load(f)
print("✅ Sample entry:\n", sample[0])


File exists: True
✅ Sample entry:
 {'id': 0, 'text': "[{'lang': 'main', 'text': 'Véritable pâte à tartiner noisettes chocolat noir'}\n {'lang': 'fr', 'text': 'Véritable pâte à tartiner noisettes chocolat noir'}] (Bovetti): 0 kcal per 100 g | Protein 0 g | Fat 0 g | Sugar 0 g. Category: Petit-déjeuners,Produits à tartiner,Produits à tartiner sucrés,Pâtes à tartiner,Pâtes à tartiner aux noisettes,Pâtes à tartiner au chocolat,Pâtes à tartiner aux noisettes et au cacao. Nutri-Score: e. Ingredients: []"}


In [25]:
!pip install sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json


In [27]:
data_path = "/content/drive/MyDrive/Colab Notebooks/food_knowledge.json"

with open(data_path, "r") as f:
    food_data = json.load(f)

corpus = [item["text"] for item in food_data]
print(f"Loaded {len(corpus)} food entries for retrieval.")


Loaded 35840 food entries for retrieval.


In [28]:
model = SentenceTransformer("all-MiniLM-L6-v2")  # fast + accurate

embeddings = model.encode(corpus, convert_to_tensor=False, show_progress_bar=True)
embeddings = np.array(embeddings, dtype="float32")

print(" Embeddings created. Shape:", embeddings.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1120 [00:00<?, ?it/s]

✅ Embeddings created. Shape: (35840, 384)


In [29]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print(f"✅ FAISS index built with {index.ntotal} entries.")


✅ FAISS index built with 35840 entries.


In [30]:
def search_food(query, top_k=3):
    query_vec = model.encode([query], convert_to_tensor=False)
    D, I = index.search(np.array(query_vec, dtype="float32"), k=top_k)
    return [corpus[i] for i in I[0]]


In [31]:
queries = [
    "high protein breakfast",
    "low sugar drinks",
    "low fat snacks",
    "healthy dinner ideas"
]

for q in queries:
    print(f"\n🔎 Query: {q}")
    for r in search_food(q, top_k=2):
        print("→", r[:200], "...")



🔎 Query: high protein breakfast
→ [{'lang': 'main', 'text': 'Instant oatmeal blueberries & cream'}
 {'lang': 'en', 'text': 'Instant oatmeal blueberries & cream'}] (Quaker): 0 kcal per 100 g | Protein 0 g | Fat 0 g | Sugar 0 g. Categor ...
→ [{'lang': 'main', 'text': 'Breakfast Grain Parfait'}
 {'lang': 'en', 'text': 'Breakfast Grain Parfait'}] (Taylor Farms): 0 kcal per 100 g | Protein 0 g | Fat 0 g | Sugar 0 g. Category: undefined. Nutr ...

🔎 Query: low sugar drinks
→ [{'lang': 'main', 'text': 'ميرندا حمضيات'}
 {'lang': 'ar', 'text': 'ميرندا حمضيات'}
 {'lang': 'en', 'text': 'Mirinda (Citrus)'}] (PepsiCo Inc.): 0 kcal per 100 g | Protein 0 g | Fat 0 g | Sugar 0 g. C ...
→ [{'lang': 'main', 'text': 'Cola soda'} {'lang': 'fr', 'text': 'Cola soda'}
 {'lang': 'en', 'text': 'Cola soda'}] (Pepsi): 0 kcal per 100 g | Protein 0 g | Fat 0 g | Sugar 0 g. Category: . Nutri-Score: ...

🔎 Query: low fat snacks
→ [{'lang': 'main', 'text': 'Advanced nutrition 100 calorie snacks drizzled crisps'}
 {

In [34]:
faiss.write_index(index, "/content/drive/MyDrive/Colab Notebooks/food_faiss.index")
np.save("/content/drive/MyDrive/Colab Notebooks/food_embeddings.npy", embeddings)
# Retriever index and embeddings saved to Drive


In [40]:
!pip install transformers datasets scikit-learn matplotlib

from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments
)
from sklearn.model_selection import train_test_split
import torch, json, math, numpy as np, matplotlib.pyplot as plt


In [42]:
# Load your cleaned JSON knowledge base (created in Day 1)
path = "/content/drive/MyDrive/Colab Notebooks/food_knowledge.json"
with open(path) as f:
    data = json.load(f)

texts = [d["text"] for d in data]

# Split 90% train / 10% test
train_texts, test_texts = train_test_split(texts, test_size=0.1, random_state=42)
print(f"Train samples: {len(train_texts)}, Test samples: {len(test_texts)}")


Train samples: 32256, Test samples: 3584


In [43]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

def tokenize_function(text_list):
    return tokenizer(
        text_list,
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_encodings = tokenize_function(train_texts)
test_encodings = tokenize_function(test_texts)


In [44]:
class FoodDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = FoodDataset(train_encodings)
test_dataset = FoodDataset(test_encodings)


In [46]:
!pip install -U transformers


In [54]:
from transformers import GPT2Tokenizer
from datasets import Dataset

# Assume you already have train_texts and test_texts as lists of strings

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

# Create raw datasets
train_dataset = Dataset.from_dict({"text": train_texts})
test_dataset = Dataset.from_dict({"text": test_texts})

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# This line is essential for GPT-2 to compute loss
train_dataset = train_dataset.map(lambda e: {"labels": e["input_ids"]})
test_dataset = test_dataset.map(lambda e: {"labels": e["input_ids"]})

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/32256 [00:00<?, ? examples/s]

Map:   0%|          | 0/3584 [00:00<?, ? examples/s]

Map:   0%|          | 0/32256 [00:00<?, ? examples/s]

Map:   0%|          | 0/3584 [00:00<?, ? examples/s]

In [55]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,1.476900
200,1.066600
300,0.994000
400,0.975800
500,0.922600
600,0.885300
700,0.879600
800,0.841700
900,0.840200
1000,0.775900
